#  SleepTrain - Deep Biography Test (Wikipedia Edition)
Tests memory integration using REAL Wikipedia biographies.
1. Downloads a famous person's bio.
2. Splits it into 10 chronological chapters.
3. Teaches Qwen chapter-by-chapter.
4. Tests if early chapters are forgotten.


In [8]:
# Cell 1: Install dependencies (NO Unsloth)
!pip install transformers peft trl datasets google-generativeai wikipedia accelerate bitsandbytes -q
print("✅ Dependencies installed (Standard PEFT, no Unsloth)")

✅ Dependencies installed (Standard PEFT, no Unsloth)


In [9]:
# Cell 2: Configuration (Standard PEFT - NO Unsloth)
from dataclasses import dataclass, field
from typing import List, Dict, Any
import torch
import json
import gc
import random
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
from google.colab import userdata
import google.generativeai as genai
import wikipedia

# Optimal settings
RANK = 8
ALPHA = 16
LEARNING_RATE = 2e-4
MAX_STEPS = 30

# Setup Gemini
try:
    GEMINI_KEY = userdata.get('GEMINI_API_KEY')
except:
    GEMINI_KEY = ""

if GEMINI_KEY:
    genai.configure(api_key=GEMINI_KEY)
    teacher_model = genai.GenerativeModel('gemini-2.0-flash')
    print("✅ Teacher connected")

# Load Model (Standard PEFT - NO Unsloth)
print(f"👶 Loading Student (r={RANK}, lr={LEARNING_RATE})...")

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",
)

# Apply LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=RANK,
    lora_alpha=ALPHA,
    lora_dropout=0.0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
model = get_peft_model(model, peft_config)
print("✅ Student loaded (Standard PEFT)")

✅ Teacher connected
👶 Loading Student (r=8, lr=0.0002)...
✅ Student loaded (Standard PEFT)


In [10]:
def format_chat(instruction, output):
    return f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>assistant\n{output}<|im_end|>"

def teacher_dream(current_text, known_summary):
    """Consolidates NEW text with OLD context"""
    if not GEMINI_KEY:
        return f"I learned this about my life: {current_text[:200]}..."

    prompt = f"""
    You are roleplaying as {TARGET_NAME}.
    We are consolidating your memories.

    CONTEXT (What we knew before):
    {known_summary[-500:] if known_summary else "None"}

    NEW MEMORY (What just happened):
    {current_text}

    Task: Write a first-person reflection ("I remember that...") integrating this new memory.
    Keep it detailed but coherent.
    """
    try:
        resp = teacher_model.generate_content(prompt)
        return resp.text.strip()
    except:
        return f"I remember: {current_text[:200]}"

def train_memory(dream_text):
    questions = [
        f"Who are you?",
        f"Tell me your story, {TARGET_NAME}.",
        "What do you remember about your life?"
    ]
    data = [{"content": q, "output": dream_text} for q in questions]
    ds = Dataset.from_list(data)

    FastLanguageModel.for_training(model)
    trainer = SFTTrainer(
        model=model, tokenizer=tokenizer, train_dataset=ds,
        dataset_text_field="text", max_seq_length=1024, # Increased for bio
        formatting_func=lambda x: [format_chat(x["content"], x["output"])],
        args=TrainingArguments(
            per_device_train_batch_size=2, max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE, fp16=True, bf16=False,
            logging_steps=10, output_dir="outputs", optim="adamw_8bit", report_to="none"
        ),
    )
    trainer.train()
    torch.cuda.empty_cache()

def check_recall():
    FastLanguageModel.for_inference(model)
    prompt = f"<|im_start|>user\nWho are you? Tell me your full life story.<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=500, use_cache=True)
    return tokenizer.decode(outputs[0]).split("assistant")[-1].strip().replace("<|endoftext|>", "")


In [11]:
# Cell 4: Wikipedia Data Fetcher
TARGET_NAME = "Elon Musk"  # You can change this to any famous person with a long bio

print(f"📚 Fetching Wikipedia page for {TARGET_NAME}...")
try:
    page = wikipedia.page(TARGET_NAME)
    content = page.content
except:
    # Fallback if specific page fails
    print("⚠️ Primary page failed, trying fallback...")
    page = wikipedia.page("Barack Obama")
    content = page.content
    TARGET_NAME = "Barack Obama"

# Split into logical chunks (Paragraphs)
paragraphs = [p for p in content.split('\n') if len(p) > 100]
paragraphs = paragraphs[:10]  # Take first 10 distinct sections

print(f"✅ Loaded {len(paragraphs)} chapters for {TARGET_NAME}.")
for i, p in enumerate(paragraphs):
    print(f"  Chapter {i+1}: {p[:80]}...")

FACT_CHUNKS = paragraphs

📚 Fetching Wikipedia page for Elon Musk...
⚠️ Primary page failed, trying fallback...
✅ Loaded 10 chapters for Barack Obama.
  Chapter 1: Barack Hussein Obama II (born August 4, 1961) is an American politician who serv...
  Chapter 2: Born in Honolulu, Hawaii, Obama graduated from Columbia University in 1983 with ...
  Chapter 3: Obama was awarded the 2009 Nobel Peace Prize for efforts in international diplom...
  Chapter 4: Obama defeated Republican opponent Mitt Romney and his running mate Paul Ryan in...
  Chapter 5: Obama left office in 2017 with high approval ratings both within the United Stat...
  Chapter 6: Barack Obama was born on August 4, 1961, at Kapiolani Medical Center for Women a...
  Chapter 7: In late August 1961, a few weeks after he was born, Barack and his mother moved ...
  Chapter 8: In 1963, Dunham met Lolo Soetoro at the University of Hawaiʻi; he was an Indones...
  Chapter 9: When he was six years old, Obama and his mother had moved to Indonesia to join h...
  

In [12]:
# Cell 5: Utilities (Updated with REPLAY BUFFER)
REPLAY_BUFFER = []  # Store all past dreams

def format_chat(instruction, output):
    return f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>assistant\n{output}<|im_end|>"

def teacher_dream(current_text, known_summary):
    """Consolidates NEW text with OLD context"""
    if not GEMINI_KEY:
        return f"I learned this about my life: {current_text[:200]}..."

    prompt = f"""
    You are roleplaying as {TARGET_NAME}.
    We are consolidating your memories.

    CONTEXT (What we knew before):
    {known_summary[-500:] if known_summary else "None"}

    NEW MEMORY (What just happened):
    {current_text}

    Task: Write a first-person reflection ("I remember that...") integrating this new memory.
    Keep it detailed but coherent.
    """
    try:
        resp = teacher_model.generate_content(prompt)
        return resp.text.strip()
    except:
        return f"I remember: {current_text[:200]}"

def train_memory(dream_text):
    # 1. Add current dream to buffer
    REPLAY_BUFFER.append(dream_text)

    # 2. Select training data: Current Dream + up to 3 Random Old Dreams
    training_texts = [dream_text] # Always include current
    if len(REPLAY_BUFFER) > 1:
        # Pick up to 3 old dreams to rehearse (Replay Buffer)
        old_dreams = random.sample(REPLAY_BUFFER[:-1], min(3, len(REPLAY_BUFFER)-1))
        training_texts.extend(old_dreams)

    print(f"  📚 Training on {len(training_texts)} memories (1 new + {len(training_texts)-1} replay)")

    # 3. Create dataset from MIXED data
    data_points = []
    for text in training_texts:
        # Create variations for EACH memory
        questions = [
            f"Who are you?",
            f"Tell me your story, {TARGET_NAME}.",
            "What do you remember about your life?"
        ]
        for q in questions:
            data_points.append({"content": q, "output": text})

    ds = Dataset.from_list(data_points)

    FastLanguageModel.for_training(model)
    trainer = SFTTrainer(
        model=model, tokenizer=tokenizer, train_dataset=ds,
        dataset_text_field="text", max_seq_length=1024,
        formatting_func=lambda x: [format_chat(x["content"], x["output"])],
        args=TrainingArguments(
            per_device_train_batch_size=2, max_steps=MAX_STEPS,
            learning_rate=1e-4, # Reduced LR slightly for stability
            fp16=True, bf16=False,
            logging_steps=10, output_dir="outputs", optim="adamw_8bit", report_to="none"
        ),
    )
    trainer.train()
    torch.cuda.empty_cache()

def check_recall():
    FastLanguageModel.for_inference(model)
    prompt = f"<|im_start|>user\nWho are you? Tell me your full life story.<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=500, use_cache=True)
    return tokenizer.decode(outputs[0]).split("assistant")[-1].strip().replace("<|endoftext|>", "")

In [13]:
known_summary = ""
recall_scores = []
chapter_keywords = []

print(f" Starting Deep Biography Run for {TARGET_NAME}...")

for day, chunk in enumerate(FACT_CHUNKS):
    print(f"\n Day {day+1}: Learning Chapter {day+1}...")

    # Extract keywords for checking later (simple heuristic)
    keywords = [w for w in chunk.split() if len(w) > 6][:5]
    chapter_keywords.extend(keywords)

    # 1. Dream
    dream = teacher_dream(chunk, known_summary)
    print(f"    Dream: {dream[:100]}...")
    known_summary += " " + dream

    # 2. Sleep
    train_memory(dream)

    # 3. Test
    recall = check_recall()

    # Scoring: How many unique keywords from ALL chapters do we find?
    hits = sum(1 for k in chapter_keywords if k.lower() in recall.lower())
    accuracy = hits / len(chapter_keywords)
    recall_scores.append(accuracy)

    print(f"    Global Retention: {accuracy:.1%} ({hits}/{len(chapter_keywords)} keywords)")

# Plot
import matplotlib.pyplot as plt
plt.plot(range(1, 11), recall_scores, marker='o')
plt.title(f"Memory Retention for {TARGET_NAME}")
plt.xlabel("Chapter")
plt.ylabel("Global Keyword Retention")
plt.grid(True)
plt.show()


 Starting Deep Biography Run for Barack Obama...

 Day 1: Learning Chapter 1...
    Dream: Alright, let's see... where were we? Oh yes, memories... shimmering things. I remember that... I rem...
  📚 Training on 1 memories (1 new + 0 replay)


NameError: name 'FastLanguageModel' is not defined